<a href="https://colab.research.google.com/github/Kalabras/2024/blob/main/re_no.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

import cv2
import pandas
from skimage import io
import matplotlib
import regex
from google.colab import drive
drive.mount('/content/drive')

def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)

def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  display_str_heights = [font.getbbox(ds)[3] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    bbox = font.getbbox(display_str)
    text_width, text_height = bbox[2], bbox[3]
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin

def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_png(img, channels=3)
  return img

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [107]:

# For running inference on the TF-Hub module.
import tensorflow as tf

import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time

# Print Tensorflow version
print(tf.__version__)

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

2.15.0
The following GPU devices are available: 


In [108]:

folder = "/content/drive/MyDrive/IndabaX2024/train/train"
import os
pattern = r"ID_(\d+)"
ids = []
def extract_ids(images_folder):
  ids = []
  for filename in os.listdir(images_folder):
    id = filename.split('.')[0]
    sub = re.findall(pattern, id)
    image_path = folder +"/"+filename
    iml = [int(sub[0]), image_path]
    im = tuple(iml)
    ids.append(im)
    ids.sort()
  return ids


In [109]:
print(extract_ids(folder))

[(0, '/content/drive/MyDrive/IndabaX2024/train/train/ID_000000.png'), (1, '/content/drive/MyDrive/IndabaX2024/train/train/ID_000001.png'), (2, '/content/drive/MyDrive/IndabaX2024/train/train/ID_000002.png'), (3, '/content/drive/MyDrive/IndabaX2024/train/train/ID_000003.png'), (4, '/content/drive/MyDrive/IndabaX2024/train/train/ID_000004.png'), (5, '/content/drive/MyDrive/IndabaX2024/train/train/ID_000005.png'), (6, '/content/drive/MyDrive/IndabaX2024/train/train/ID_000006.png'), (7, '/content/drive/MyDrive/IndabaX2024/train/train/ID_000007.png'), (8, '/content/drive/MyDrive/IndabaX2024/train/train/ID_000008.png'), (9, '/content/drive/MyDrive/IndabaX2024/train/train/ID_000009.png'), (10, '/content/drive/MyDrive/IndabaX2024/train/train/ID_000010.png'), (11, '/content/drive/MyDrive/IndabaX2024/train/train/ID_000011.png'), (12, '/content/drive/MyDrive/IndabaX2024/train/train/ID_000012.png'), (13, '/content/drive/MyDrive/IndabaX2024/train/train/ID_000013.png'), (14, '/content/drive/MyDrive/

In [110]:
# prompt: import regex and use it to get the image with her id

# import re

# # Define the regular expression to match the image ID
# pattern = r"ID_(\d+)"
# images = extract_ids(folder)
# # Initialize the image path variable
# image_path = None
# ids = []
# # Iterate through the list of image IDs
# for image_id in images:
#   sub = re.findall(pattern, image_id)
#   ids.append(int(sub[0]))

# print(ids)


In [111]:
from google.colab.patches import cv2_imshow

test = pandas.read_csv("/content/drive/MyDrive/IndabaX2024/Train.csv")
test_df = pandas.DataFrame(test)
for index, elem in enumerate(ids):
  id = elem[0]
  if index == 2:
    break
  for index, row in test_df.iterrows():
    class_id = row["ToothClass"]
    if (row["Image_ID"] == id):
      xmin = row["Xmin"]
      ymin = row["Ymin"]
      xmax = row["Xmax"]
      ymax = row["Ymax"]
      image = cv2.imread(elem[1])
      plt.imshow(image)
      plt.show()
      cv2.rectangle(elem[1], (xmin, ymin), (xmax, ymax), (36,255,12), 1)
      cv2.putText(image, class_id, (xmin, ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
      plt.imshow(image)
      plt.show()

In [112]:
import matplotlib.pyplot as plt
import cv2  # Assuming you have OpenCV installed (`!pip install opencv-python`)

test = pandas.read_csv("/content/drive/MyDrive/IndabaX2024/Train.csv")
test_df = pandas.DataFrame(test)
print("Hi, MOM")
# Assuming `ids` is a list containing tuples (image_id, image_path)
for index, elem in enumerate(ids):
  id, image_path = elem  # Unpack the tuple elements for clarity
  print(image_path)
  image = cv2.imread(image_path)  # Read the image using OpenCV
  # Display the image with bounding box and class label using Matplotlib
  plt.imshow(image)
  for index, row in test_df.iterrows():
    class_id = row["ToothClass"]
    if row["Image_ID"] == id:
      xmin = row["Xmin"]
      ymin = row["Ymin"]
      xmax = row["Xmax"]
      ymax = row["Ymax"]
      cv2.rectangle(image, (xmin, ymax), (xmax, ymin), (0, 255, 0), 2)
      # Add a title
      cv2.putText(image, class_id, (xmin, ymax - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

      # Display the image
      plt.imshow(image)
      plt.show()

Hi, MOM


In [113]:
import matplotlib.pyplot as plt
import cv2  # Assuming you have OpenCV installed (`!pip install opencv-python`)
extract_ids(folder)

test = pandas.read_csv("/content/drive/MyDrive/IndabaX2024/Train.csv")
test_df = pandas.DataFrame(test)
ids=[]
extract_ids(folder)
# Assuming `ids` is a list containing tuples (image_id, image_path)
for index, elem in enumerate(ids):
  id, image_path = elem  # Unpack the tuple elements

  print(f"Processing image: {image_path}")  # Print image path for verification

  image = cv2.imread(image_path)  # Read the image using OpenCV

  for index, row in test_df.iterrows():
    class_id = str(row["ToothClass"])
    if row["Image_ID"] == id:
      xmin = row["Xmin"]
      ymin = row["Ymin"]
      xmax = row["Xmax"]
      ymax = row["Ymax"]

      # Draw the bounding box (corrected coordinates)
      plt.plot([xmin, xmax], [ymin, ymin], color='green', linewidth=2)  # Top line
      plt.plot([xmin, xmax], [ymax, ymax], color='green', linewidth=2)  # Bottom line
      plt.plot([xmin, xmin], [ymin, ymax], color='green', linewidth=2)  # Left line
      plt.plot([xmax, xmax], [ymin, ymax], color='green', linewidth=2)  # Right line

      # Add class label (adjusted position for better visibility)
      plt.text(xmin + 5, ymax - 5, class_id, color='white', fontsize=12,
               bbox=dict(facecolor='green', alpha=0.5))  # Green semi-transparent background

      plt.axis('off')  # Hide axes for cleaner display
      plt.imshow(image)
      plt.show()


In [114]:
# img = io.imread("/content/drive/MyDrive/IndabaX2024/train/train/ID_000000.png")

# io.imshow(img)

In [115]:
# prompt: putting a bounding box with a title on an image then showing it

# Load the image
image = cv2.imread("/content/drive/MyDrive/IndabaX2024/train/train/ID_000000.png")
test = pandas.read_csv("/content/drive/MyDrive/IndabaX2024/Train.csv")
test_df = pandas.DataFrame(test)
for index, elem in enumerate(ids):
  # if index > 2:
  #   break
  id, image_path = elem  # Unpack the tuple elements
  print(f"Processing image: {image_path}")  # Print image path for verification
  image = cv2.imread(image_path)  # Read the image using OpenCV
  # Create a bounding box
  class_id = str(row["ToothClass"])
  left = row["Xmin"]
  bottom = row["Ymin"]
  right = row["Xmax"]
  top = row["Ymax"]
  # top, bottom, left, right = 10, 100, 10, 100
  cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)

  # Add a title
  cv2.putText(image, class_id, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

  # Display the image
  plt.imshow(image)
  plt.show()


In [116]:
test = pandas.read_csv("/content/drive/MyDrive/IndabaX2024/Train.csv")
test

,Image_ID,ToothClass,Xmin,Ymin,Xmax,Ymax
0,0,33,284,315,325,361
1,0,37,372,119,448,193
2,0,47,40,114,113,186
3,0,35,325,236,379,287
4,0,41,200,362,237,411
...,...,...,...,...,...,...
16008,1199,13,337,318,388,366
16009,1199,12,309,339,349,389
16010,1199,22,177,352,218,400
16011,1199,24,111,288,174,343
